# Problem 4.1 - Image alignment using PCA

In [1]:
import os
import cv2 as cv
import numpy as np
from PIL import Image

In [2]:
def rotation_angle(img_path):
    image = cv.imread(img_path)
    image_grayscale = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image_binary = cv.threshold(image_grayscale, 50, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
    image_contours = cv.findContours(image_binary, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)[0]

    area_threshold = 10000
    largest_connected_area = 0
    largest_contour = None
    for contour in image_contours:
        area = cv.contourArea(contour)
        if area < area_threshold and area > largest_connected_area:
            largest_connected_area = area
            largest_contour = contour

    ROI_coords = largest_contour[:, -1]
    m_x = np.mean(ROI_coords, axis=0)
    P_x = ((ROI_coords - m_x).T @ (ROI_coords - m_x)) / (ROI_coords.shape[0] - 1)

    eigen_values, eigen_vectors = np.linalg.eig(P_x)
    sortedIndices = np.argsort(eigen_values)[::-1]
    eigen_values = eigen_values[sortedIndices]
    eigen_vectors = eigen_vectors[:, sortedIndices]

    theta = np.rad2deg(np.arctan2(eigen_vectors[1, 0], eigen_vectors[0, 0]))
    if theta > 90:
        theta = 180 - theta

    return theta
template_img_path = "./Data/4.1/Template_Input.png"
template_img = Image.open(template_img_path)
template_img_angle = rotation_angle(template_img_path)

test_images_path = "./Data/4.1/KIMIA-99/"
test_images_out_path = "./Data/4.1/KIMIA-99-Aligned/"
test_images = os.listdir(test_images_path)

for test_image in test_images:
    test_image_path = os.path.join(test_images_path, test_image)
    test_image_angle = rotation_angle(test_image_path) - template_img_angle

    intial_test_image = Image.open(test_image_path)
    rotated_test_image = intial_test_image.rotate(test_image_angle)
    
    rotated_test_image.save(os.path.join(test_images_out_path, test_image))

<center><img src="./Data/4.1/Flowchart.png" alt="Flowchart" />